In [14]:
from pathlib import Path
import numpy as np
import gensim

base_path = Path("go-basic")

In [15]:
wv = gensim.models.KeyedVectors.load(str(base_path / "owl2vec" / "ontology.embeddings"), mmap="r")

[11/14/24 09:46:34] INFO     loading KeyedVectors object from go-basic/owl2vec/ontology.embeddings     ]8;id=389943;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=878097;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py#481\481]8;;\

                    INFO     loading wv recursively from go-basic/owl2vec/ontology.embeddings.wv.*     ]8;id=502373;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=236614;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py#515\515]8;;\
                             with mmap=r                                                                           

                    INFO     loading vectors from go-basic/owl2vec/ontology.embeddings.wv.vectors.npy  ]8;id=699035;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=672310;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py#520\520]8;;\
                             with mmap=r                                                                           

                    INFO     loading syn1neg from go-basic/owl2vec/ontology.embeddings.syn1neg.npy     ]8;id=326123;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=927893;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py#520\520]8;;\
                             with mmap=r                                                                           

                    INFO     setting ignored attribute cum_table to None                               ]8;id=831334;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=447700;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py#553\553]8;;\

[11/14/24 09:46:36] INFO     Word2Vec lifecycle event {'fname':                                        ]8;id=365916;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=818884;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py#447\447]8;;\
                             'go-basic/owl2vec/ontology.embeddings', 'datetime':                                   
                             '2024-11-14T09:46:36.638646', 'gensim': '4.3.3', 'python': '3.9.20 |                  
                             packaged by conda-forge | (main, Sep 30 2024, 17:48:00) \n[Clang 17.0.6               
                             ]', 'platform': 'macOS-15.1-arm64-arm-64bit', 'event': 'loaded'}                      

In [16]:
import logging
from rich.logging import RichHandler

logging.basicConfig(level="INFO", format="%(message)s", handlers=[RichHandler()])

from pathlib import Path
import numpy as np
import gensim
from nltk import word_tokenize
import re

# Load the model
wv = gensim.models.KeyedVectors.load(str(base_path / "owl2vec" / "ontology.embeddings"), mmap="r")

# Helper functions from OWL2Vec_Plus.py
def pre_process_words(words):
    text = ' '.join([re.sub(r'https?:\/\/.*[\r\n]*', '', word, flags=re.MULTILINE) for word in words])
    tokens = word_tokenize(text)
    processed_tokens = [token.lower() for token in tokens if token.isalpha()]
    return processed_tokens

n_s = []
def embed(model, instances, uri_label=None):
    def word_embeding(inst):
        v = np.zeros(model.vector_size)
        if uri_label and inst in uri_label:
            words = uri_label.get(inst)
            n = 0
            for word in words:
                if word in model.wv.index_to_key:
                    v += model.wv.get_vector(word)
                    n += 1
            return (v / n if n > 0 else v, n)
        else:
            return (v, 0)

    feature_vectors = []
    for instance in instances:
        # Get both URI and word embeddings
        # v_uri = model.get_vector(instance) if instance in model.index_to_key else np.zeros(model.vector_size)
        v_word, n = word_embeding(inst=instance)
        n_s.append(n)
        # Concatenate them
        feature_vectors.append(v_word)

    return feature_vectors

# Load classes and annotations
classes = [line.strip() for line in open(base_path / "split" / "classes.txt").readlines()]
uri_label = {}
for line in open(base_path / "cache" / "annotations.txt").readlines():
    tmp = line.strip().split()
    if tmp[1] == "preferred_label":
        if tmp[0] not in uri_label:
            uri_label[tmp[0]] = []
        uri_label[tmp[0]] += pre_process_words(tmp[2:])

# Calculate embeddings
classes_e = embed(model=wv, instances=classes, uri_label=uri_label)

logging.info(f"Average number of words per class: {np.mean(n_s)}")
# Extract GO IDs from class URIs and create dictionary
go_ids = [cls.split('/')[-1] for cls in classes]
embeddings_dict = {go_id: emb for go_id, emb in zip(go_ids, classes_e)}

# Save embeddings
np.save(str(base_path / "owl2vec" / "ontology.embeddings.npy"), embeddings_dict)

print(f"Saved embeddings for {len(embeddings_dict)} classes")
print(f"Each embedding has dimension: {len(next(iter(embeddings_dict.values())))} (URI + word embeddings)")

                    INFO     loading KeyedVectors object from go-basic/owl2vec/ontology.embeddings     ]8;id=517363;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=782103;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py#481\481]8;;\

                    INFO     loading wv recursively from go-basic/owl2vec/ontology.embeddings.wv.*     ]8;id=942659;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=412640;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py#515\515]8;;\
                             with mmap=r                                                                           

                    INFO     loading vectors from go-basic/owl2vec/ontology.embeddings.wv.vectors.npy  ]8;id=467842;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=519836;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py#520\520]8;;\
                             with mmap=r                                                                           

                    INFO     loading syn1neg from go-basic/owl2vec/ontology.embeddings.syn1neg.npy     ]8;id=944294;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=890140;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py#520\520]8;;\
                             with mmap=r                                                                           

                    INFO     setting ignored attribute cum_table to None                               ]8;id=144864;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=534869;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py#553\553]8;;\

[11/14/24 09:46:38] INFO     Word2Vec lifecycle event {'fname':                                        ]8;id=194766;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=11145;file:///Users/mmtf/p/research/contempro/owl2vec_star/.pixi/envs/default/lib/python3.9/site-packages/gensim/utils.py#447\447]8;;\
                             'go-basic/owl2vec/ontology.embeddings', 'datetime':                                   
                             '2024-11-14T09:46:38.377182', 'gensim': '4.3.3', 'python': '3.9.20 |                  
                             packaged by conda-forge | (main, Sep 30 2024, 17:48:00) \n[Clang 17.0.6               
                             ]', 'platform': 'macOS-15.1-arm64-arm-64bit', 'event': 'loaded'}                      

[11/14/24 09:46:44] INFO     Average number of words per class: 4.110164888457808                   ]8;id=893832;file:///var/folders/pb/_0h8x45x091307x0pwzykw8h0000gn/T/ipykernel_68782/755403242.py\755403242.py]8;;\:]8;id=734822;file:///var/folders/pb/_0h8x45x091307x0pwzykw8h0000gn/T/ipykernel_68782/755403242.py#61\61]8;;\

Saved embeddings for 51550 classes
Each embedding has dimension: 100 (URI + word embeddings)


In [10]:
uri_label

{'http://purl.obolibrary.org/obo/GO_0005623': ['obsolete',
  'cell',
  'cell',
  'and',
  'encapsulating',
  'structures'],
 'http://purl.obolibrary.org/obo/IAO_0000115': ['definition'],
 'http://purl.obolibrary.org/obo/IAO_0000233': ['term', 'tracker', 'item'],
 'http://purl.obolibrary.org/obo/IAO_0000700': ['has',
  'ontology',
  'root',
  'term'],
 'http://purl.obolibrary.org/obo/IAO_0100001': ['term', 'replaced', 'by'],
 'http://purl.obolibrary.org/obo/go#syngo_official_label': ['label',
  'approved',
  'by',
  'the',
  'syngo',
  'project'],
 'http://purl.obolibrary.org/obo/go#systematic_synonym': ['systematic',
  'synonym'],
 'http://www.geneontology.org/formats/oboInOwl#SubsetProperty': [],
 'http://www.geneontology.org/formats/oboInOwl#SynonymTypeProperty': [],
 'http://www.geneontology.org/formats/oboInOwl#consider': ['consider'],
 'http://www.geneontology.org/formats/oboInOwl#hasAlternativeId': [],
 'http://www.geneontology.org/formats/oboInOwl#hasBroadSynonym': [],
 'http://

In [17]:
base_path

PosixPath('go-basic')